##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - 與 OneTwo 整合

這本筆記本展示了如何通過 [OneTwo](https://github.com/google-deepmind/onetwo) 使用 Gemma。OneTwo 是一個由 Google DeepMind 發佈的 Python 函式庫，旨在簡化與 LLMs 的互動。

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemma-cookbook/blob/zh-tw/Gemma/Integrate_with_OneTwo.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


## 設定

### 選擇 Colab 執行環境
要完成這個指南，你需要有一個具有足夠資源的 Colab 執行環境來執行 Gemma 模型。在這種情況下，你可以使用 T4 GPU:

1. 在 Colab 視窗的右上角，選擇 **▾ (Additional connection options)** 。
2. 選擇 **Change runtime type** 。
3. 在 **Hardware accelerator** 下，選擇 **T4 GPU** 。

### Gemma 設定

要完成這個指南，你首先需要完成 [Gemma setup](https://ai.google.dev/gemma/docs/setup) 的設定說明。Gemma 設定說明會告訴你如何完成以下步驟:

* 在 [Kaggle](kaggle.com) 上獲取 Gemma 的訪問權限。
* 生成和配置 [Kaggle username and API key](https://www.kaggle.com/docs/api)。將它們設置為 Colab 機密 'KAGGLE_USERNAME' 和 'KAGGLE_KEY'。


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

In [ ]:
# Install kaggle
! pip install kaggle
! pip install kagglehub

In [ ]:
import kagglehub
from kagglehub import auth

auth.set_kaggle_credentials(
    username=userdata.get("KAGGLE_USERNAME"), api_key=userdata.get("KAGGLE_KEY")
)
kagglehub.login()

Kaggle credentials set.


In [ ]:
import os

VARIANT = "2b"  # @param ['2b', '1.1-2b-it', '7b', '1.1-7b-it'] {type:"string"}
weights_dir = kagglehub.model_download(f"google/gemma/Flax/{VARIANT}")

checkpoint_path = os.path.join(weights_dir, VARIANT)
vocab_path = os.path.join(weights_dir, "tokenizer.model")

## 使用 Gemma 與 OneTwo

### 安裝 OneTwo


In [ ]:
!pip install git+https://github.com/google-deepmind/onetwo

  Cloning https://github.com/google-deepmind/onetwo to /tmp/pip-req-build-zjyeelh4
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/onetwo /tmp/pip-req-build-zjyeelh4
  Resolved https://github.com/google-deepmind/onetwo to commit 43c3b67e4b479e49e31baefb304d1487b4c893bb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/google-deepmind/gemma.git to /tmp/pip-install-8hrdcipz/gemma_c35bc1f9a46943db8dce9b89f9a3baed
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/gemma.git /tmp/pip-install-8hrdcipz/gemma_c35bc1f9a46943db8dce9b89f9a3baed
  Resolved https://github.com/google-deepmind/gemma.git to commit a24194737dcb54b7392091e9ba772aea1cb68ffb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Imports


In [ ]:
from onetwo import ot
from onetwo.builtins import llm

### 註冊 Gemma 為後端


In [ ]:
from onetwo.backends import gemma_local

backend = gemma_local.Gemma(checkpoint_path=checkpoint_path, vocab_path=vocab_path)
backend.register()

### 執行推論與 `llm.generate_text()`


In [ ]:
# Ask the model to continue the given text.
e = llm.generate_text(
    "Three not so well known cities in Gemany are", max_tokens=30, temperature=0.3
)
print(ot.run(e))

 the cities of <strong>Duisburg</strong>, <strong>Cologne</strong> and <strong>Bonn</strong>.

<strong>Duisburg</strong> is a city
